In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Term Rewriting System for Regular Expressions

In this notebook, we implement a **Term Rewriting System** to simplify complex regular expressions. The expressions generated by algorithms like *State Elimination* often contain redundancies (e.g., $R + \emptyset$, $\varepsilon \cdot R$).

We define an algebraic simplification engine based on axioms of **Regular Algebra** (Kleene Algebra).

### Abstract Syntax Tree (AST) Definition
Since we want to keep this notebook self-contained and avoid circular dependencies with previous course materials, we define a local recursive type `MyRegExp`.

* **Atoms:** `string` (variables or characters) or `number` ($0$ for $\emptyset$).
* **Unary:** A Tuple `[RegExp, '*']` representing the Kleene Star $R^*$.
* **Binary:** A Tuple `[RegExp, op, RegExp]` representing Union ($+$) and Concatenation ($\cdot$).

In [ ]:
import { RecursiveSet, Tuple } from "recursive-set";

// === LOCAL TYPES ===
type LocalBinaryOp = '⋅' | '+';
type LocalUnaryOp = '*';

// Recursive Definition of the Regular Expression AST
type MyRegExp = 
  | number      
  | string      
  | Tuple<[MyRegExp, LocalUnaryOp]>             
  | Tuple<[MyRegExp, LocalBinaryOp, MyRegExp]>; 

type Subst = Map<string, MyRegExp>;
type Rule = [MyRegExp, MyRegExp];

### Type Guards and Helper Functions

To ensure type safety within the rewriting engine, we implement **Type Guards**. These functions allow the system to narrow down the specific variant of the recursive `MyRegExp` union type (Atom, Binary, or Unary).

We also define a helper function `T(...)`. This is a shorthand constructor for creating `Tuple` objects, which makes defining the algebraic rules much more readable later on.

In [ ]:
// === TYPE GUARDS ===

function isAtom(r: MyRegExp): r is number | string {
    return typeof r === 'number' || typeof r === 'string';
}

/**
 * Checks if a string is a Variable (A-Z) used in pattern matching rules.
 */
function isVariable(r: MyRegExp): r is string {
    return typeof r === 'string' && r.length === 1 && r >= 'A' && r <= 'Z';
}

// Checks if it is a Kleene Star Tuple: [RegExp, '*']
function isKleene(r: MyRegExp): r is Tuple<[MyRegExp, LocalUnaryOp]> {
    return r instanceof Tuple && r.length === 2 && r.values[1] === '*';
}

// Checks if it is a Binary Operation: [RegExp, Op, RegExp]
function isBinary(r: MyRegExp): r is Tuple<[MyRegExp, LocalBinaryOp, MyRegExp]> {
    return r instanceof Tuple && r.length === 3;
}

// === HELPER FUNCTION T (Type-safe via Overloads) ===

// 1. Signature: Unary (Kleene)
function T(arg1: MyRegExp, op: LocalUnaryOp): MyRegExp;
// 2. Signature: Binary (Concat/Union)
function T(arg1: MyRegExp, op: LocalBinaryOp, arg3: MyRegExp): MyRegExp;
// Implementation
function T(...args: (MyRegExp | string)[]): MyRegExp {
    // We cast to MyRegExp because 'new Tuple' is generic, 
    // but we know the result fits our recursive schema.
    return new Tuple(...args) as unknown as MyRegExp;
}

### Pattern Matching Engine

The core of a rewriting system is **Pattern Matching**. We need to determine if a specific term (e.g., `(a + 0)`) matches a defined rule pattern (e.g., `(R + 0)`).

1.  **`deepEquals`**: Recursively checks if two ASTs are structurally identical.
2.  **`match`**: Checks if a `term` matches a `pattern`. If the pattern contains variables (like `R` or `S`), it binds the corresponding sub-term to the variable in the `substitution` map.
3.  **`apply`**: Reconstructs the term by replacing the variables in the Right-Hand-Side (RHS) of a rule with their bound values.

In [ ]:
// === MATCHING LOGIC ===

function deepEquals(a: MyRegExp, b: MyRegExp): boolean {
    if (a === b) return true;
    if (isAtom(a) || isAtom(b)) return a === b;
    
    // Recursive structure check
    if (isKleene(a) && isKleene(b)) {
        return deepEquals(a.values[0], b.values[0]);
    }

    if (isBinary(a) && isBinary(b)) {
        return a.values[1] === b.values[1] && // Operator check
               deepEquals(a.values[0], b.values[0]) && // Left
               deepEquals(a.values[2], b.values[2]);   // Right
    }

    return false;
}

function match(pattern: MyRegExp, term: MyRegExp, substitution: Subst): boolean {
    // 1. Variable Match (e.g. pattern="R")
    if (isVariable(pattern)) {
        if (substitution.has(pattern)) {
            // Variable already bound, check if current term equals previous binding
            return deepEquals(substitution.get(pattern)!, term);
        } else {
            // New binding
            substitution.set(pattern, term);
            return true;
        }
    }

    // 2. Primitives Match
    if (isAtom(pattern) || isAtom(term)) {
        return pattern === term;
    }

    // 3. Tuple Match (Structural Recursion)
    if (isKleene(pattern) && isKleene(term)) {
        return match(pattern.values[0], term.values[0], substitution);
    }

    if (isBinary(pattern) && isBinary(term)) {
        if (pattern.values[1] !== term.values[1]) return false;
        return match(pattern.values[0], term.values[0], substitution) &&
               match(pattern.values[2], term.values[2], substitution);
    }

    return false;
}

function apply(term: MyRegExp, substitution: Subst): MyRegExp {
    if (isVariable(term)) {
        if (substitution.has(term)) {
            return substitution.get(term)!; 
        }
        return term;
    }

    if (isAtom(term)) {
        return term;
    }

    if (isKleene(term)) {
        const inner = apply(term.values[0], substitution);
        return new Tuple(inner, term.values[1]) as unknown as MyRegExp;
    }

    if (isBinary(term)) {
        const left = apply(term.values[0], substitution);
        const right = apply(term.values[2], substitution);
        return new Tuple(left, term.values[1], right) as unknown as MyRegExp;
    }

    return term;
}

function rewrite(term: MyRegExp, rule: Rule): { simplified: boolean, result: MyRegExp } {
    const [lhs, rhs] = rule;
    const substitution: Subst = new Map();

    if (match(lhs, term, substitution)) {
        return { simplified: true, result: apply(rhs, substitution) };
    } else {
        return { simplified: false, result: term };
    }
}

### Algebraic Rules (Axioms)

Here we define the **Axioms of Regular Algebra**. These rules are applied to reduce the complexity of the expressions.

Common rules include:
* **Identity Laws:** $R + 0 = R$, $\varepsilon \cdot R = R$
* **Annihilation:** $R \cdot 0 = 0$
* **Kleene Star Simplifications:** $\varepsilon + R \cdot R^* = R^*$ (based on Arden's Rule lemmas)
* **Associativity:** $(R + S) + T = R + (S + T)$

We use the helper `T(...)` to write these rules in a compact and readable format.

In [ ]:
// === THE RULES ===

function getRules(): Rule[] {
    const rules: Rule[] = [
        // Addition (Identity & Idempotence)
        [T('R', '+', 0), 'R'],
        [T(0, '+', 'R'), 'R'],
        [T('R', '+', 'R'), 'R'],

        // Kleene Star & Epsilon Simplifications
        [T('ε', '+', T('R', '*')), T('R', '*')],
        [T(T('R', '*'), '+', 'ε'), T('R', '*')],
        [T('ε', '+', T('R', '⋅', T('R', '*'))), T('R', '*')],
        [T('ε', '+', T(T('R', '*'), '⋅', 'R')), T('R', '*')],
        [T(T('R', '⋅', T('R', '*')), '+', 'ε'), T('R', '*')],
        [T(T(T('R', '*'), '⋅', 'R'), '+', 'ε'), T('R', '*')],

        // Distributive Laws (Arden's Rule specifics)
        [T('S', '+', T('S', '⋅', 'T')), T('S', '⋅', T('ε', '+', 'T'))],
        [T('S', '+', T('T', '⋅', 'S')), T(T('ε', '+', 'T'), '⋅', 'S')],

        // Multiplication (Annihilator & Identity)
        [T(0, '⋅', 'R'), 0],
        [T('R', '⋅', 0), 0],
        [T('ε', '⋅', 'R'), 'R'],
        [T('R', '⋅', 'ε'), 'R'],

        // Absorption
        [T(T('ε', '+', 'R'), '⋅', T('R', '*')), T('R', '*')],
        [T(T('R', '+', 'ε'), '⋅', T('R', '*')), T('R', '*')],
        [T(T('R', '*'), '⋅', T('R', '+', 'ε')), T('R', '*')],
        [T(T('R', '*'), '⋅', T('ε', '+', 'R')), T('R', '*')],

        // Constant Kleene Stars
        [T(0, '*'), 'ε'],
        [T('ε', '*'), 'ε'],
        
        // Nested Kleene Stars
        [T(T('ε', '+', 'R'), '*'), T('R', '*')],
        [T(T('R', '+', 'ε'), '*'), T('R', '*')],

        // Associativity (Rebalancing to the right)
        [T('R', '+', T('S', '+', 'T')), T(T('R', '+', 'S'), '+', 'T')],
        [T('R', '⋅', T('S', '⋅', 'T')), T(T('R', '⋅', 'S'), '⋅', 'T')],
        
        // Complex Absorption
        [T(T('R', '⋅', T('S', '*')), '⋅', T('ε', '+', 'S')), T('R', '⋅', T('S', '*'))]
    ];
    return rules;
}

### Main Simplification Algorithm

We implement a **Fixpoint Iteration** algorithm:

1.  `simplifyOnce`: Traverses the AST tree and tries to apply *any* matching rule to *any* node (or its children).
2.  `simplify`: Calls `simplifyOnce` repeatedly until the term stops changing (a fixpoint is reached) or a safety limit is hit to prevent infinite loops.

This ensures that rewriting applied to a sub-term (like $0^* \to \varepsilon$) can trigger further simplifications in the parent term (like $R \cdot \varepsilon \to R$).

In [ ]:
function simplifyOnce(term: MyRegExp, rules: Rule[]): MyRegExp {
    if (isAtom(term)) return term;

    // 1. Try to rewrite current node
    for (const rule of rules) {
        const { simplified, result } = rewrite(term, rule);
        if (simplified) {
            return result;
        }
    }

    // 2. Recurse into children if no rule matched at this level
    if (isKleene(term)) {
        const newInner = simplifyOnce(term.values[0], rules);
        return new Tuple(newInner, '*') as unknown as MyRegExp;
    }

    if (isBinary(term)) {
        const newLeft = simplifyOnce(term.values[0], rules);
        const newRight = simplifyOnce(term.values[2], rules);
        return new Tuple(newLeft, term.values[1], newRight) as unknown as MyRegExp;
    }

    return term;
}

/**
 * MAIN FUNCTION: Fixpoint iteration.
 */
function simplify(t: MyRegExp): MyRegExp {
    const rules = getRules();
    let current = t;
    
    let iterations = 0;
    const MAX_ITERATIONS = 1000; 

    while (true) {
        const next = simplifyOnce(current, rules);
        
        if (deepEquals(current, next)) {
            return next;
        }
        
        current = next;
        iterations++;
        if (iterations > MAX_ITERATIONS) {
            console.warn("Simplification limit reached. Possible cycle.");
            return current;
        }
    }
}

### Pretty Printing

Finally, we need a way to read the result. `regexpToString` converts the internal recursive `Tuple` structure (AST) back into a standard human-readable string format (e.g., `a(b)*`).

In [ ]:
// === STRING OUTPUT ===
function regexpToString(r: MyRegExp): string {
    if (r === 0) return "0";
    if (r === "ε" || r === "𝜀") return "𝜀";
    if (typeof r === 'string') return r;
    if (typeof r === 'number') return r.toString();

    if (isKleene(r)) {
        const inner = r.values[0];
        const sInner = regexpToString(inner);

        if (isAtom(inner)) {
            return sInner + "*";
        } else {
            return "(" + sInner + ")*";
        }
    }

    if (isBinary(r)) {
        const left = r.values[0];
        const op = r.values[1];
        const right = r.values[2];

        const s1 = regexpToString(left);
        const s2 = regexpToString(right);

        if (op === "⋅") {
            return s1 + s2; 
        }

        if (op === "+") {
            return "(" + s1 + "+" + s2 + ")";
        }
    }

    return JSON.stringify(r);
}